# MATH&ML-8 Алгоритмы на основе деревьев решений.
###  Содержание <a class="anchor" id=0></a>

- [1. Введение](#1)
- [2. Деревья решений (регрессия и классификация)](#2)
- [3. Алгоритм CART. Параметры](#3)
- [4. Алгоритм CART. Неоднородность](#4)
- [5. Алгоритм CART. Рекурсия](#5)
- [6. Алгоритм CART. Значимость](#6)
- [7. Деревья решений. Практика](#7)
- [8. Итоги](#8)

# 1. Введение <a class="anchor" id=1></a>

[к содержанию](#0)

<img src=m8_img1.png width=600>

# 2. Деревья решений (регрессия и классификация) <a class="anchor" id=2></a>

[к содержанию](#0)

У нас есть набор из 500 объектов, которые описываются пятью какими-то характеристиками. Например, это могут быть 500 пациентов, у которых измерили пять медицинских параметров: вес, рост, давление, количество лейкоцитов в крови и уровень сахара.

В паре с объектом всегда идёт целевая переменная, которую необходимо научиться предсказывать. Например, целевой переменной может быть признак болезни: 1 — болен диабетом, 0 — не болен диабетом.

Наша цель — построить модель машинного обучения, которая на основе характеристик объекта предсказывает целевую переменную. Для нашего примера это будет модель, которая на основе медицинских параметров выдаёт свой прогноз о наличии у пациента диабета.

Это была классическая постановка задачи машинного обучения в свободной форме. Давайте теперь посмотрим, как будет выглядеть та же самая задача на языке математики.

>Примечание. Далее для простоты изложения и упрощения обозначений мы не будем писать стрелки над векторами. Если какая-то переменная будет являться векторной, мы будем указывать на это в явном виде.
>
>Постановка задачи машинного обучения может выглядеть по-разному. Например, когда мы разбирали линейные модели в модулях по линейной алгебре, мы давали постановку задачи на языке линейной алгебры. От нас требовалось линейно выразить целевой вектор $y$ через векторы-факторы $x_j$. 
>
>В контексте разговора о деревьях нам будет удобнее использовать не чистый аппарат линейной алгебры, а совместить его с теорией множеств.

Пусть задано обучающее множество пар объектов и ответов к ним $Q=\{(x,\ y)\}$, где $x\ \in \ X$ — векторы из $\mathbb{R}^M$, описываемые $M$ координатами (признаками), $y\ \in \ Y$ — целевая метка объекта, $|Q|=N$. Множество $Y$ может быть непрерывным (задача регрессии) с мощность $|Y|=N$ или дискретным (задача классификации) с мощностью $|Y|=K$, где $K$ — количество классов.

В примере выше:

* $x = (x_1, x_2, x_3, x_4, x_5) — один отдельный пациент;
* $x_j$ — признак (фактор), описывающий его;
* $X$ — множество всех пациентов;
* $N=500$ — количество пациентов;
* $M=5$ — количество признаков, описывающих пациентов;
* $Y={0,1}$ — множество уникальных классов объектов;
* $K=2$ — количество классов (мощность множества $Y$).

**Наша цель** — составить такую функцию/модель $f(x)$, которая наилучшим образом определяет зависимость между векторами $x$ из множества $X$ и целевой переменной $y$ из множества $Y$ — $f:X\rightarrow Y$.

Строго говоря, такую функцию мы будем искать не в аналитическом виде, как мы делали, например, в случае линейной регрессии, а в виде алгоритма, то есть в виде последовательности действий. Обычно в математике алгоритм обозначается как $a(x)$ или $a:X \rightarrow Y$.

Алгоритм $a(x)$ мы будем искать в семействе деревьев решений.

## ОПРЕДЕЛЕНИЕ РЕШАЮЩЕГО ДЕРЕВА

>Если говорить простыми словами, **обученное дерево решений** — это последовательность вопросов, которые задаются поступающим на его вход объектам.

В зависимости от того, выполняется ли заданное в вопросе условие, объект переходит к одному из следующих вопросов, и так происходит до тех пор, пока не будет получен конечный результат — прогноз дерева решений для данного объекта. Всё это очень напоминает стандартные рассуждения по принципу `«если…, то»`.

Пройдя путь, который зависит от характеристик объекта, через последовательность от первого вопроса (корня дерева) до финального ответа (листа) мы можем определить значение целевой переменной. В задаче регрессии финальный ответ дерева — некоторое число, а в задаче классификации — класс объекта.

<img src=m8_img2.png width=600>

Давайте переведём вышесказанное на более формальный язык математики, на котором, собственно, и работает программная реализация дерева решений.

>Дерево решений представляет собой последовательность условий вида $x_j \leq t$, которые называются **предикатами**, или **решающими правилами**.

Формально сами предикаты в вершинах мы ранее обозначали как:

$B_{\nu }(x_j,\ t)\ =\ I[x_j\le t],$

где $v$ — номер вершины графа, а $I$ с квадратными скобками $[]$ — обозначение индикаторной функции (она равна $1$ (`True`), если условие внутри скобок выполняется, и $0$ (`False`) — в противном случае).

>**Примечание**. Далее для удобства мы будем обозначать индикаторную функцию $I$ простыми квадратными скобками, например:

$B_{\nu }(x_j,\ t)\ =\ [x_j\le t]$

Каждый предикат разделяет множество объектов $X$ на два подмножества:

* левое — множество тех объектов, для которых заданное в предикате условие выполняется: $X^{left}=\{x\ |\ x_j\le t\}$;
* правое — множество тех объектов, для которых условие ложно: $X^{right} = X \backslash X^{left} = \{x \mid x_j > t \}$.

Последовательность предикатов лучше всего представлять в виде **ациклического связного графа**. 

Давайте посмотрим на пример такого графа для дерева решений глубиной 4. Дерево решает задачу классификации 15 наблюдений (мощность множества $X$) на три класса (мощность множества $Y$):

<img src=m8_img3.png width=600>

>Примечание. Информация, указанная в вершинах графа, определяется во время обучения дерева решений, о процедуре которого мы поговорим в следующих юнитах:
>
>$x_j\le t$ — условие предиката;
>$gini$ — значение критерия информативности (в данном случае используется критерий Джини);
>$samples$ — число объектов из обучающей выборки, попавших в вершину;
>$value$ — число объектов каждого из классов из обучающей выборки, попавших в вершину.

В построенном графе выделяют $три типа вершин$:

* корневая вершина (root node) — то, откуда всё начинается;
* внутренние вершины (intermediate nodes);
* листья (leafs) — конечные вершины дерева, в которых определяется конечный «ответ» — прогноз дерева решений.

<img src=m8_img4.png width=600>

Вершины графа группируются в уровни, которые называются **глубиной дерева (depth)**. Отсчёт уровней ведётся с 0 (то есть корневая вершина не считается при подсчёте глубины дерева).

>Корневая и внутренние вершины содержат предикаты $B_{\nu }(x_j,\ t)$, на основе которых организуется движение по графу. В ходе предсказания осуществляется проход по дереву к некоторому листу. Для каждого объекта из выборки движение начинается из корня.

В вершине $v$ проход осуществляется влево, если для объекта $x$ условие, записанное в предикате, выполняется, то есть $B_{\nu }(x_j,\ t)=1$, и вправо, если условие является для объекта #x# ложным, то есть $B_{\nu }(x_j,\ t)=0$. Проход продолжается до момента, пока будет достигнут некоторый лист. Ответом алгоритма для объекта $x$ считается прогноз $\hat{y}=a(x)$, приписанный этому листу.

| Регрессия | Классификация|
| - | - |
| В задаче **регрессии** прогноз целевой переменной $\hat{y}$ будет определяться принципом усреднения, то есть объекту $x$ в качестве прогноза присваивается среднее (или медианное) значение целевой переменной по объектам, попавшим в лист на этапе обучения дерева. | В задаче **классификации** прогноз целевой переменной $\hat{y}$ будет определяться принципом голосования большинства, то есть объекту $x$ присваивается самый популярный класс объектов, попавших в лист на этапе обучения дерева. |

# 3. Алгоритм CART. Параметры <a class="anchor" id=3></a>

[к содержанию](#0)

Схематично поэтапное построение дерева решений выглядит следующим образом:

| Картинка | Описание |
| - | - |
| <img src=m8_img5.png>  |  Допустим, мы построили решающий пень. В результате построения получилась одна родительская и две дочерние вершины, каждой из которых соответствует своя часть исходной выборки. |
| <img src=m8_img6.png>  |  Чтобы построить следующий уровень дерева, необходимо проделать ту же процедуру построения решающего пня, но теперь уже каждая из дочерних вершин воспринимается как родительская, и её основе строится новый решающий пень. В результате получится дерево глубиной 2. |
| <img src=m8_img7.png>  |  Строим следующий уровень: дочерние вершины снова воспринимаем как родительские и на их основе строим новые решающие пни. Получаем дерево глубиной 3.  Повторяем эти действия, пока не дойдём до некоторого предела, на котором нужно будет остановиться: например, если мы достигли идеальной однородности (все объекты относятся к одному классу). |

  /

>**Примечание**. Формально такой последовательный алгоритм построения дерева относится к категории жадных алгоритмов. 

Внимательные студенты уже догадались, что описанная нами процедура построения дерева является рекурсией. Мы рекурсивно используем алгоритм построения решающего пня для каждой из дочерних вершин, тем самым создавая всё новые и новые вершины графа.

Теперь давайте переведём наши рассуждения на язык математики и сформулируем итоговый алгоритм построения дерева.

>**Примечание**. Отметим, что, так как мы теперь рассматриваем не решающие пни, а деревья произвольной глубины, то в наших рассуждениях появится **индекс вершины дерева** $v$. Он вводится для того, чтобы обозначить, что та или иная величина рассчитывается для конкретной вершины.

Итак, пусть $Q_v={(x,y)}$ — множество объектов и ответов к ним, попавших в вершину с номером $v$, $N_v=|Q_v|$. Пусть задан критерий информативности $H(Q)$, который зависит от задачи (регрессия — `Squared Error`, `Absolute Error` или `Poisson Error`, классификация — `Gini` или `Entropy`). Также задан некоторый критерий остановки рекурсии $stop \ criterion(Q_v)$. О том, какими могут быть критерии остановки, мы поговорим чуть позже.

Наш **алгоритм построения дерева решений `CART`** будет выглядеть следующим образом:

1. Создаём вершину под номером $v$.

2. По умолчанию ветвление дерева решений прекращается, когда достигается однородность, то есть $H(Q)=0$. 

    **2.1.  Если условие остановки выполнено.**
    Формируем листовую вершину $v$ и ставим в соответствие этому листу ответ, который будет выдаваться для новых объектов, которые дойдут до этого листа. Назовём эту часть `create_leaf()`.
    
    * В случае задачи регрессии ответ вычисляется как среднее (если критерий информативности $H(Q)$ — `Squared Error` или `Poisson Error`) или медиана (если критерий информативности — `Absolute Error`):

    $${\hat{y}}_v=\frac{1}{N_v}\sum_{y\ \in \ Q_v}{}y$$

    $${\hat{y}}_v=median_{y\ \in \ Q_v}(y)$$

    * В случае задачи классификации ответ вычисляется как метка самого популярного среди объектов $X_v$ класса или как оценки вероятностей принадлежности $P_{vk}$ к каждому из классов:

    $$P_{vk}=\frac{1}{N_v}\sum_{y\ \in \ Y_v}{}[y=k]$$

    $${\hat{y}}_v=argmax_{k\in K}(P_{vk})$$

    **2.2. Если условие остановки не выполнено.**

    Строим решающий пень. Формируем решающее правило $B_{v}(x_j,\ t)$ для вершины $v$. Из возможных комбинаций $w=(j, t)$ выбираем такую, которая определяет наилучшее разбиение текущего множества объектов и ответов к ним $Q_v$ на две части: $Q^{left}=\{(x, y)|x_j\leq t\}$ — левую, для которой условие предиката истинно, и $Q^{right}_v=\{(x,\ y) \mid \ x_j >t\}$ — правую, для которой условие предиката ложно:

    $$G(Q,\ w)=\frac{N^{left}_v}{N_v}H(Q^{left}_v)+\frac{N^{right}_v}{N_v}H(Q^{right}_v)$$

    $$w_{opt}=(j_{opt},\ t_{opt})=argmin_w\ G(Q,\ w)$$

    $$B_{\nu }(x_j,\ t)=[x_{j_{opt}}\le t_{opt}]$$

    Эту часть алгоритма мы обозначали ранее как `best_split()`.

    В результате разбиения будут созданы два подмножества объектов — $Q^{left}_v$ и $Q^{rigth}_v$. На основе этих подмножеств мы с помощью рекурсии создадим две новые вершины дерева (левую и правую) и свяжем их с текущей вершиной.

    Для выборок $Q^{left}_v$ и $Q^{rigth}_v$ процедура будет повторяться рекурсивно, пока не выполнится критерий остановки.

3. Возвращаем созданную вершину.

## Файл `DecisionTree_CART.ipynb`

# 4. Алгоритм CART. Неоднородность <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Алгоритм CART. Рекурсия <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Алгоритм CART. Значимость <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Деревья решений. Практика <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Итоги <a class="anchor" id=8></a>

[к содержанию](#0)